In [1]:
from tensorflow import keras
import tensorflow as tf 
# Common imports
import os

from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Flatten
from tensorflow.python.keras.layers import Dense, Dropout
from tensorflow.python.keras.layers import Activation
from tensorflow.keras.layers import BatchNormalization
import matplotlib.pyplot as plt

from tensorflow.python.framework import ops
from tensorflow.python.keras import backend_config
from tensorflow.python.keras.optimizer_v2 import learning_rate_schedule
from tensorflow.python.keras.optimizer_v2 import optimizer_v2
from tensorflow.python.ops import array_ops
from tensorflow.python.ops import control_flow_ops
from tensorflow.python.ops import math_ops
from tensorflow.python.ops import state_ops
from tensorflow.python.ops import variables as tf_variables
from tensorflow.python.util.tf_export import keras_export

import scipy
import numpy as np

2023-01-10 22:00:24.852632: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-01-10 22:00:24.852665: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
def parameter(n):
    alpha = []
    for i in range(n):
    	alpha.append(1.2 - 0.05**(i+1)) #0.8 
    return alpha

def parameter1(n):
    alpha = []
    for i in range(n):
        alpha.append(1.4 - 0.005**(i+1)) # 0.8
    beta = []
    for i in range(n):
        beta.append(0.9 - 0.005**(i+1)) #0.8
    return alpha, beta

def summation(n):
    summ = 0
    alpha = parameter(n)
    for i in range(n):
        summ += alpha[i]
    return summ

def Fisher(n):
    FIM = []
    alpha = parameter(n)
    for i in range(n):
        Row = []
        for j in range(n):
            if i == j: 
                Row.append(special.polygamma(1, alpha[i]) - special.polygamma(1, summation(n)))
            else:
                Row.append(- special.polygamma(1, summation(n)))
        FIM.append(Row)
    FIM = numpy.array(FIM)
    FIM = numpy.linalg.inv(FIM)
    return FIM

def Fisher2(n):
    FIM = []
    for i in range(n):
        Zero = []
        for j in range(n):
            Zero.append(0)
        FIM.append(Zero)
    alpha, beta = parameter1(n)
    for i in range(0,n,2):
        FIM[i][i] = special.polygamma(1, alpha[i]) - special.polygamma(1, alpha[i]+beta[i])
        FIM[i+1][i] = - special.polygamma(1, alpha[i]+beta[i])
        FIM[i][i+1] = - special.polygamma(1, alpha[i]+beta[i])
        FIM[i+1][i+1] = special.polygamma(1, beta[i]) - special.polygamma(1, alpha[i]+beta[i])
    FIM = numpy.array(FIM)
    FIM = numpy.linalg.inv(FIM)
    return FIM

In [3]:
def Riemann_Liouville_derivative(gradient, alpha, kappa, variable):
    RL_gradient = np.array([])
    for i in range(len(gradient)):
        RL_gradient.append(
            1/scipy.special.gamma(len(gradient)-alpha)*scipy.misc.derivative(
                scipy.integrate.quad(
                    lambda tau: variable[i]/(tau-variable[i])**(alpha-kappa+1),0,variable[i])
                , n = kappa)
            )
    return RL_gradient

In [4]:
def Caputo_derivative(gradient, alpha, kappa, variable):
    C_gradient = np.array([])
    for i in range(len(gradient)):
        C_gradient.append(
            1/scipy.special.gamma(len(gradient)-alpha)*scipy.integrate.quad(
                lambda tau: scipy.misc.derivative(variable[i], n = kappa)/(tau-variable[i])**(alpha-kappa+1),0,variable[i])
            )
    return C_gradient

In [5]:
def Grunwald_Letnikov_derivative(gradient, h, alpha, kappa, variable):
    GL_gradient = np.array([])
    for i in range(len(gradient)):
        summ = 0
        for j in range(len(gradient)):
            summ += (-1)**k * scipy.special.gamma(alpha+1)*variable[j]/(scipy.special.gamma(kappa+1)*scipy.special.gamma(alpha-kappa+1))
        GL_gradient.append(
            1/h**alpha * summ
        ) 
    return GL_gradient

In [6]:
@keras_export("keras.optimizers.SGD")
class SGD(optimizer_v2.OptimizerV2):

  _HAS_AGGREGATE_GRAD = True

  def __init__(self,
               learning_rate=0.01,
               momentum=0.0,
               nesterov=False,
               name="SGD",
               **kwargs):
    super(SGD, self).__init__(name, **kwargs)
    self._set_hyper("learning_rate", kwargs.get("lr", learning_rate))
    self._set_hyper("decay", self._initial_decay)

    self._momentum = False
    if isinstance(momentum, tf.Tensor) or callable(momentum) or momentum > 0:
      self._momentum = True
    if isinstance(momentum, (int, float)) and (momentum < 0 or momentum > 1):
      raise ValueError(f"`momentum` must be between [0, 1]. Received: "
                       f"momentum={momentum} (of type {type(momentum)}).")
    self._set_hyper("momentum", momentum)

    self.nesterov = nesterov

  def _create_slots(self, var_list):
    if self._momentum:
      for var in var_list:
        self.add_slot(var, "momentum")

  def _prepare_local(self, var_device, var_dtype, apply_state):
    super(SGD, self)._prepare_local(var_device, var_dtype, apply_state)
    apply_state[(var_device, var_dtype)]["momentum"] = tf.identity(
        self._get_hyper("momentum", var_dtype))

  def _resource_apply_dense(self, grad, var, apply_state=None):
    var_device, var_dtype = var.device, var.dtype.base_dtype
    coefficients = ((apply_state or {}).get((var_device, var_dtype))
                    or self._fallback_apply_state(var_device, var_dtype))

    if self._momentum:
      momentum_var = self.get_slot(var, "momentum")
      return tf.raw_ops.ResourceApplyKerasMomentum(
          var=var.handle,
          accum=momentum_var.handle,
          lr=coefficients["lr_t"],
          grad=grad,
          momentum=coefficients["momentum"],
          use_locking=self._use_locking,
          use_nesterov=self.nesterov)
    else:
      return tf.raw_ops.ResourceApplyGradientDescent(
          var=var.handle,
          alpha=coefficients["lr_t"],
          delta=grad,
          use_locking=self._use_locking)

  def _resource_apply_sparse_duplicate_indices(self, grad, var, indices,
                                               **kwargs):
    if self._momentum:
      return super(SGD, self)._resource_apply_sparse_duplicate_indices(
          grad, var, indices, **kwargs)
    else:
      var_device, var_dtype = var.device, var.dtype.base_dtype
      coefficients = (kwargs.get("apply_state", {}).get((var_device, var_dtype))
                      or self._fallback_apply_state(var_device, var_dtype))

      return tf.raw_ops.ResourceScatterAdd(
          resource=var.handle,
          indices=indices,
          updates=-grad * coefficients["lr_t"])

  def _resource_apply_sparse(self, grad, var, indices, apply_state=None):
    # This method is only needed for momentum optimization.
    var_device, var_dtype = var.device, var.dtype.base_dtype
    coefficients = ((apply_state or {}).get((var_device, var_dtype))
                    or self._fallback_apply_state(var_device, var_dtype))

    momentum_var = self.get_slot(var, "momentum")
    return tf.raw_ops.ResourceSparseApplyKerasMomentum(
        var=var.handle,
        accum=momentum_var.handle,
        lr=coefficients["lr_t"],
        grad=grad,
        indices=indices,
        momentum=coefficients["momentum"],
        use_locking=self._use_locking,
        use_nesterov=self.nesterov)

  def get_config(self):
    config = super(SGD, self).get_config()
    config.update({
        "learning_rate": self._serialize_hyperparameter("learning_rate"),
        "decay": self._initial_decay,
        "momentum": self._serialize_hyperparameter("momentum"),
        "nesterov": self.nesterov,
    })
    return config

In [7]:
class RLFGD(optimizer_v2.OptimizerV2):
  _HAS_AGGREGATE_GRAD = True

  def __init__(self,
               learning_rate=0.01,
               momentum=0.0,
               nesterov=False,
               name="RLFGD",
               **kwargs):
    super(RLFGD, self).__init__(name, **kwargs)
    self._set_hyper("learning_rate", kwargs.get("lr", learning_rate))
    self._set_hyper("decay", self._initial_decay)

    self._momentum = False
    if isinstance(momentum, tf.Tensor) or callable(momentum) or momentum > 0:
      self._momentum = True
    if isinstance(momentum, (int, float)) and (momentum < 0 or momentum > 1):
      raise ValueError(f"`momentum` must be between [0, 1]. Received: "
                       f"momentum={momentum} (of type {type(momentum)}).")
    self._set_hyper("momentum", momentum)

    self.nesterov = nesterov

  def _create_slots(self, var_list):
    if self._momentum:
      for var in var_list:
        self.add_slot(var, "momentum")

  def _prepare_local(self, var_device, var_dtype, apply_state):
    super(RLFGD, self)._prepare_local(var_device, var_dtype, apply_state)
    apply_state[(var_device, var_dtype)]["momentum"] = tf.identity(
        self._get_hyper("momentum", var_dtype))

  def _resource_apply_dense(self, grad, var, apply_state=None):
    var_device, var_dtype = var.device, var.dtype.base_dtype
    coefficients = ((apply_state or {}).get((var_device, var_dtype))
                    or self._fallback_apply_state(var_device, var_dtype))

    if self._momentum:
      momentum_var = self.get_slot(var, "momentum")
      return tf.raw_ops.ResourceApplyKerasMomentum(
          var=var.handle,
          accum=momentum_var.handle,
          lr=coefficients["lr_t"],
          grad=grad,
          momentum=coefficients["momentum"],
          use_locking=self._use_locking,
          use_nesterov=self.nesterov)
    else:
      return tf.raw_ops.ResourceApplyGradientDescent(
          var=var.handle,
          alpha=coefficients["lr_t"],
          delta=grad,
          use_locking=self._use_locking)

  def _resource_apply_sparse_duplicate_indices(self, grad, var, indices,
                                               **kwargs):
    if self._momentum:
      return super(RLFGD, self)._resource_apply_sparse_duplicate_indices(
          grad, var, indices, **kwargs)
    else:
      var_device, var_dtype = var.device, var.dtype.base_dtype
      coefficients = (kwargs.get("apply_state", {}).get((var_device, var_dtype))
                      or self._fallback_apply_state(var_device, var_dtype))

      return tf.raw_ops.ResourceScatterAdd(
          resource=var.handle,
          indices=indices,
          updates=np.dot(Fisher(len(grad)),Riemann_Liouville_derivative(grad, 7.8, 2, var).transpose()) * coefficients["lr_t"])
          #updates=-Riemann_Liouville_derivative(grad, 8.8, 4, var) * coefficients["lr_t"])
          #updates=-grad * coefficients["lr_t"])

  def _resource_apply_sparse(self, grad, var, indices, apply_state=None):
    # This method is only needed for momentum optimization.
    var_device, var_dtype = var.device, var.dtype.base_dtype
    coefficients = ((apply_state or {}).get((var_device, var_dtype))
                    or self._fallback_apply_state(var_device, var_dtype))

    momentum_var = self.get_slot(var, "momentum")
    return tf.raw_ops.ResourceSparseApplyKerasMomentum(
        var=var.handle,
        accum=momentum_var.handle,
        lr=coefficients["lr_t"],
        grad=grad,
        indices=indices,
        momentum=coefficients["momentum"],
        use_locking=self._use_locking,
        use_nesterov=self.nesterov)

  def get_config(self):
    config = super(RLFGD, self).get_config()
    config.update({
        "learning_rate": self._serialize_hyperparameter("learning_rate"),
        "decay": self._initial_decay,
        "momentum": self._serialize_hyperparameter("momentum"),
        "nesterov": self.nesterov,
    })
    return config

In [8]:
class CFGD(optimizer_v2.OptimizerV2):
  _HAS_AGGREGATE_GRAD = True

  def __init__(self,
               learning_rate=0.01,
               momentum=0.0,
               nesterov=False,
               name="CFGD",
               **kwargs):
    super(CFGD, self).__init__(name, **kwargs)
    self._set_hyper("learning_rate", kwargs.get("lr", learning_rate))
    self._set_hyper("decay", self._initial_decay)

    self._momentum = False
    if isinstance(momentum, tf.Tensor) or callable(momentum) or momentum > 0:
      self._momentum = True
    if isinstance(momentum, (int, float)) and (momentum < 0 or momentum > 1):
      raise ValueError(f"`momentum` must be between [0, 1]. Received: "
                       f"momentum={momentum} (of type {type(momentum)}).")
    self._set_hyper("momentum", momentum)

    self.nesterov = nesterov

  def _create_slots(self, var_list):
    if self._momentum:
      for var in var_list:
        self.add_slot(var, "momentum")

  def _prepare_local(self, var_device, var_dtype, apply_state):
    super(CFGD, self)._prepare_local(var_device, var_dtype, apply_state)
    apply_state[(var_device, var_dtype)]["momentum"] = tf.identity(
        self._get_hyper("momentum", var_dtype))

  def _resource_apply_dense(self, grad, var, apply_state=None):
    var_device, var_dtype = var.device, var.dtype.base_dtype
    coefficients = ((apply_state or {}).get((var_device, var_dtype))
                    or self._fallback_apply_state(var_device, var_dtype))

    if self._momentum:
      momentum_var = self.get_slot(var, "momentum")
      return tf.raw_ops.ResourceApplyKerasMomentum(
          var=var.handle,
          accum=momentum_var.handle,
          lr=coefficients["lr_t"],
          grad=grad,
          momentum=coefficients["momentum"],
          use_locking=self._use_locking,
          use_nesterov=self.nesterov)
    else:
      return tf.raw_ops.ResourceApplyGradientDescent(
          var=var.handle,
          alpha=coefficients["lr_t"],
          delta=grad,
          use_locking=self._use_locking)

  def _resource_apply_sparse_duplicate_indices(self, grad, var, indices,
                                               **kwargs):
    if self._momentum:
      return super(CFGD, self)._resource_apply_sparse_duplicate_indices(
          grad, var, indices, **kwargs)
    else:
      var_device, var_dtype = var.device, var.dtype.base_dtype
      coefficients = (kwargs.get("apply_state", {}).get((var_device, var_dtype))
                      or self._fallback_apply_state(var_device, var_dtype))

      return tf.raw_ops.ResourceScatterAdd(
          resource=var.handle,
          indices=indices,
          updates=np.dot(Fisher(len(grad)),Caputo_derivative(grad, 9.8, 2, var).transpose()) * coefficients["lr_t"])
          #updates=-Caputo_derivative(grad, 8.8, 4, var) * coefficients["lr_t"])
          #updates=-grad * coefficients["lr_t"])

  def _resource_apply_sparse(self, grad, var, indices, apply_state=None):
    # This method is only needed for momentum optimization.
    var_device, var_dtype = var.device, var.dtype.base_dtype
    coefficients = ((apply_state or {}).get((var_device, var_dtype))
                    or self._fallback_apply_state(var_device, var_dtype))

    momentum_var = self.get_slot(var, "momentum")
    return tf.raw_ops.ResourceSparseApplyKerasMomentum(
        var=var.handle,
        accum=momentum_var.handle,
        lr=coefficients["lr_t"],
        grad=grad,
        indices=indices,
        momentum=coefficients["momentum"],
        use_locking=self._use_locking,
        use_nesterov=self.nesterov)

  def get_config(self):
    config = super(CFGD, self).get_config()
    config.update({
        "learning_rate": self._serialize_hyperparameter("learning_rate"),
        "decay": self._initial_decay,
        "momentum": self._serialize_hyperparameter("momentum"),
        "nesterov": self.nesterov,
    })
    return config

In [9]:
class GLFGD(optimizer_v2.OptimizerV2):
  _HAS_AGGREGATE_GRAD = True

  def __init__(self,
               learning_rate=0.01,
               momentum=0.0,
               nesterov=False,
               name="GLFGD",
               **kwargs):
    super(GLFGD, self).__init__(name, **kwargs)
    self._set_hyper("learning_rate", kwargs.get("lr", learning_rate))
    self._set_hyper("decay", self._initial_decay)

    self._momentum = False
    if isinstance(momentum, tf.Tensor) or callable(momentum) or momentum > 0:
      self._momentum = True
    if isinstance(momentum, (int, float)) and (momentum < 0 or momentum > 1):
      raise ValueError(f"`momentum` must be between [0, 1]. Received: "
                       f"momentum={momentum} (of type {type(momentum)}).")
    self._set_hyper("momentum", momentum)

    self.nesterov = nesterov

  def _create_slots(self, var_list):
    if self._momentum:
      for var in var_list:
        self.add_slot(var, "momentum")

  def _prepare_local(self, var_device, var_dtype, apply_state):
    super(GLFGD, self)._prepare_local(var_device, var_dtype, apply_state)
    apply_state[(var_device, var_dtype)]["momentum"] = tf.identity(
        self._get_hyper("momentum", var_dtype))

  def _resource_apply_dense(self, grad, var, apply_state=None):
    var_device, var_dtype = var.device, var.dtype.base_dtype
    coefficients = ((apply_state or {}).get((var_device, var_dtype))
                    or self._fallback_apply_state(var_device, var_dtype))

    if self._momentum:
      momentum_var = self.get_slot(var, "momentum")
      return tf.raw_ops.ResourceApplyKerasMomentum(
          var=var.handle,
          accum=momentum_var.handle,
          lr=coefficients["lr_t"],
          grad=grad,
          momentum=coefficients["momentum"],
          use_locking=self._use_locking,
          use_nesterov=self.nesterov)
    else:
      return tf.raw_ops.ResourceApplyGradientDescent(
          var=var.handle,
          alpha=coefficients["lr_t"],
          delta=grad,
          use_locking=self._use_locking)

  def _resource_apply_sparse_duplicate_indices(self, grad, var, indices,
                                               **kwargs):
    if self._momentum:
      return super(GLFGD, self)._resource_apply_sparse_duplicate_indices(
          grad, var, indices, **kwargs)
    else:
      var_device, var_dtype = var.device, var.dtype.base_dtype
      coefficients = (kwargs.get("apply_state", {}).get((var_device, var_dtype))
                      or self._fallback_apply_state(var_device, var_dtype))

      return tf.raw_ops.ResourceScatterAdd(
          resource=var.handle,
          indices=indices,
          #updates=-Grunwald_Letnikov_derivative(grad, 0.01, 6.8, 4, var) * coefficients["lr_t"])
          updates=np.dot(Fisher(len(grad)),Grunwald_Letnikov_derivative(grad, 0.001, 7.8, 2, var).transpose()) * coefficients["lr_t"])          
          #updates=-grad * coefficients["lr_t"])

  def _resource_apply_sparse(self, grad, var, indices, apply_state=None):
    # This method is only needed for momentum optimization.
    var_device, var_dtype = var.device, var.dtype.base_dtype
    coefficients = ((apply_state or {}).get((var_device, var_dtype))
                    or self._fallback_apply_state(var_device, var_dtype))

    momentum_var = self.get_slot(var, "momentum")
    return tf.raw_ops.ResourceSparseApplyKerasMomentum(
        var=var.handle,
        accum=momentum_var.handle,
        lr=coefficients["lr_t"],
        grad=grad,
        indices=indices,
        momentum=coefficients["momentum"],
        use_locking=self._use_locking,
        use_nesterov=self.nesterov)

  def get_config(self):
    config = super(GLFGD, self).get_config()
    config.update({
        "learning_rate": self._serialize_hyperparameter("learning_rate"),
        "decay": self._initial_decay,
        "momentum": self._serialize_hyperparameter("momentum"),
        "nesterov": self.nesterov,
    })
    return config

In [10]:
@keras_export('keras.optimizers.Adam')
class Adam(optimizer_v2.OptimizerV2):

  _HAS_AGGREGATE_GRAD = True

  def __init__(self,
               learning_rate=0.001,
               beta_1=0.9,
               beta_2=0.999,
               epsilon=1e-7,
               amsgrad=False,
               name='Adam',
               **kwargs):
    super(Adam, self).__init__(name, **kwargs)
    self._set_hyper('learning_rate', kwargs.get('lr', learning_rate))
    self._set_hyper('decay', self._initial_decay)
    self._set_hyper('beta_1', beta_1)
    self._set_hyper('beta_2', beta_2)
    self.epsilon = epsilon or backend_config.epsilon()
    self.amsgrad = amsgrad

  def _create_slots(self, var_list):
    # Create slots for the first and second moments.
    # Separate for-loops to respect the ordering of slot variables from v1.
    for var in var_list:
      self.add_slot(var, 'm')
    for var in var_list:
      self.add_slot(var, 'v')
    if self.amsgrad:
      for var in var_list:
        self.add_slot(var, 'vhat')

  def _prepare_local(self, var_device, var_dtype, apply_state):
    super(Adam, self)._prepare_local(var_device, var_dtype, apply_state)

    local_step = tf.cast(self.iterations + 1, var_dtype)
    beta_1_t = tf.identity(self._get_hyper('beta_1', var_dtype))
    beta_2_t = tf.identity(self._get_hyper('beta_2', var_dtype))
    beta_1_power = tf.pow(beta_1_t, local_step)
    beta_2_power = tf.pow(beta_2_t, local_step)
    lr = (apply_state[(var_device, var_dtype)]['lr_t'] *
          (tf.sqrt(1 - beta_2_power) / (1 - beta_1_power)))
    apply_state[(var_device, var_dtype)].update(
        dict(
            lr=lr,
            epsilon=tf.convert_to_tensor(
                self.epsilon, var_dtype),
            beta_1_t=beta_1_t,
            beta_1_power=beta_1_power,
            one_minus_beta_1_t=1 - beta_1_t,
            beta_2_t=beta_2_t,
            beta_2_power=beta_2_power,
            one_minus_beta_2_t=1 - beta_2_t))

  def set_weights(self, weights):
    params = self.weights
    # If the weights are generated by Keras V1 optimizer, it includes vhats
    # even without amsgrad, i.e, V1 optimizer has 3x + 1 variables, while V2
    # optimizer has 2x + 1 variables. Filter vhats out for compatibility.
    num_vars = int((len(params) - 1) / 2)
    if len(weights) == 3 * num_vars + 1:
      weights = weights[:len(params)]
    super(Adam, self).set_weights(weights)

  def _resource_apply_dense(self, grad, var, apply_state=None):
    var_device, var_dtype = var.device, var.dtype.base_dtype
    coefficients = ((apply_state or {}).get((var_device, var_dtype))
                    or self._fallback_apply_state(var_device, var_dtype))

    m = self.get_slot(var, 'm')
    v = self.get_slot(var, 'v')

    if not self.amsgrad:
      return tf.raw_ops.ResourceApplyAdam(
          var=var.handle,
          m=m.handle,
          v=v.handle,
          beta1_power=coefficients['beta_1_power'],
          beta2_power=coefficients['beta_2_power'],
          lr=coefficients['lr_t'],
          beta1=coefficients['beta_1_t'],
          beta2=coefficients['beta_2_t'],
          epsilon=coefficients['epsilon'],
          grad=grad,
          use_locking=self._use_locking)
    else:
      vhat = self.get_slot(var, 'vhat')
      return tf.raw_ops.ResourceApplyAdamWithAmsgrad(
          var=var.handle,
          m=m.handle,
          v=v.handle,
          vhat=vhat.handle,
          beta1_power=coefficients['beta_1_power'],
          beta2_power=coefficients['beta_2_power'],
          lr=coefficients['lr_t'],
          beta1=coefficients['beta_1_t'],
          beta2=coefficients['beta_2_t'],
          epsilon=coefficients['epsilon'],
          grad=grad,
          use_locking=self._use_locking)

  def _resource_apply_sparse(self, grad, var, indices, apply_state=None):
    var_device, var_dtype = var.device, var.dtype.base_dtype
    coefficients = ((apply_state or {}).get((var_device, var_dtype))
                    or self._fallback_apply_state(var_device, var_dtype))

    # m_t = beta1 * m + (1 - beta1) * g_t
    m = self.get_slot(var, 'm')
    m_scaled_g_values = grad * coefficients['one_minus_beta_1_t']
    m_t = tf.compat.v1.assign(m, m * coefficients['beta_1_t'],
                           use_locking=self._use_locking)
    with tf.control_dependencies([m_t]):
      m_t = self._resource_scatter_add(m, indices, m_scaled_g_values)

    # v_t = beta2 * v + (1 - beta2) * (g_t * g_t)
    v = self.get_slot(var, 'v')
    v_scaled_g_values = (grad * grad) * coefficients['one_minus_beta_2_t']
    v_t = tf.compat.v1.assign(v, v * coefficients['beta_2_t'],
                           use_locking=self._use_locking)
    with tf.control_dependencies([v_t]):
      v_t = self._resource_scatter_add(v, indices, v_scaled_g_values)

    if not self.amsgrad:
      v_sqrt = tf.sqrt(v_t)
      var_update = tf.compat.v1.assign_sub(
          var, coefficients['lr'] * m_t / (v_sqrt + coefficients['epsilon']),
          use_locking=self._use_locking)
      return tf.group(*[var_update, m_t, v_t])
    else:
      v_hat = self.get_slot(var, 'vhat')
      v_hat_t = tf.maximum(v_hat, v_t)
      with tf.control_dependencies([v_hat_t]):
        v_hat_t = tf.compat.v1.assign(
            v_hat, v_hat_t, use_locking=self._use_locking)
      v_hat_sqrt = tf.sqrt(v_hat_t)
      var_update = tf.compat.v1.assign_sub(
          var,
          coefficients['lr'] * m_t / (v_hat_sqrt + coefficients['epsilon']),
          use_locking=self._use_locking)
      return tf.group(*[var_update, m_t, v_t, v_hat_t])

  def get_config(self):
    config = super(Adam, self).get_config()
    config.update({
        'learning_rate': self._serialize_hyperparameter('learning_rate'),
        'decay': self._initial_decay,
        'beta_1': self._serialize_hyperparameter('beta_1'),
        'beta_2': self._serialize_hyperparameter('beta_2'),
        'epsilon': self.epsilon,
        'amsgrad': self.amsgrad,
    })
    return config

In [11]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

# Cast the records into float values
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
  
# normalize image pixel values by dividing 
# by 255
gray_scale = 255
x_train /= gray_scale
x_test /= gray_scale

print("Feature matrix:", x_train.shape)
print("Target matrix:", x_test.shape)
print("Feature matrix:", y_train.shape)
print("Target matrix:", y_test.shape)


Feature matrix: (60000, 28, 28)
Target matrix: (10000, 28, 28)
Feature matrix: (60000,)
Target matrix: (10000,)


In [12]:
model = Sequential([
    
     # reshape 28 row * 28 column data to 28*28 rows
    Flatten(input_shape=(28, 28)),
    
      # dense layer 1
    Dense(512, activation='relu'),
    BatchNormalization(),  
    Dropout(0.2),
    # dense layer 2
    Dense(128, activation='relu'), 
    BatchNormalization(),  
    Dropout(0.2),
      # output layer
    Dense(10, activation='softmax'), 
])

optimizer = RLFGD(learning_rate = 0.075, momentum = 0.99, nesterov = True)
model.compile(optimizer=optimizer,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

history = model.fit(x_train, y_train, epochs=20, 
          batch_size=2000, 
          validation_split=0.2)

2023-01-10 22:00:27.272883: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-01-10 22:00:27.272915: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2023-01-10 22:00:27.272935: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (ruslan-Inspiron-5770): /proc/driver/nvidia/version does not exist
2023-01-10 22:00:27.273335: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch 1/20
24/24 [==============================] - 2s 59ms/step - loss: 0.5427 - accuracy: 0.8361 - val_loss: 0.4597 - val_accuracy: 0.8708
Epoch 2/20
24/24 [==============================] - 1s 50ms/step - loss: 0.2266 - accuracy: 0.9296 - val_loss: 0.2475 - val_accuracy: 0.9273
Epoch 3/20
24/24 [==============================] - 1s 50ms/step - loss: 0.1693 - accuracy: 0.9488 - val_loss: 0.2766 - val_accuracy: 0.9315
Epoch 4/20
24/24 [==============================] - 1s 50ms/step - loss: 0.1376 - accuracy: 0.9580 - val_loss: 0.2273 - val_accuracy: 0.9466
Epoch 5/20
24/24 [==============================] - 1s 50ms/step - loss: 0.1178 - accuracy: 0.9638 - val_loss: 0.1840 - val_accuracy: 0.9564
Epoch 6/20
24/24 [==============================] - 1s 50ms/step - loss: 0.1043 - accuracy: 0.9679 - val_loss: 0.1665 - val_accuracy: 0.9603
Epoch 7/20
24/24 [==============================] - 1s 51ms/step - loss: 0.0882 - accuracy: 0.9722 - val_loss: 0.1472 - val_accuracy: 0.9647
Epoch 8/20
24

In [13]:
model = Sequential([
    
     # reshape 28 row * 28 column data to 28*28 rows
    Flatten(input_shape=(28, 28)),
    
      # dense layer 1
    Dense(512, activation='relu'),
    BatchNormalization(),  
    Dropout(0.2),
    # dense layer 2
    Dense(128, activation='relu'), 
    BatchNormalization(),  
    Dropout(0.2),
      # output layer
    Dense(10, activation='softmax'), 
])

optimizer = CFGD(learning_rate = 0.05, momentum = 0.99, nesterov = True)
model.compile(optimizer=optimizer,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

history1 = model.fit(x_train, y_train, epochs=20, 
          batch_size=2000, 
          validation_split=0.2)

Epoch 1/20
24/24 [==============================] - 2s 70ms/step - loss: 0.6109 - accuracy: 0.8130 - val_loss: 0.6413 - val_accuracy: 0.7937
Epoch 2/20
24/24 [==============================] - 1s 52ms/step - loss: 0.2451 - accuracy: 0.9250 - val_loss: 0.3093 - val_accuracy: 0.9046
Epoch 3/20
24/24 [==============================] - 1s 61ms/step - loss: 0.1831 - accuracy: 0.9447 - val_loss: 0.2520 - val_accuracy: 0.9272
Epoch 4/20
24/24 [==============================] - 1s 53ms/step - loss: 0.1481 - accuracy: 0.9550 - val_loss: 0.2260 - val_accuracy: 0.9402
Epoch 5/20
24/24 [==============================] - 1s 54ms/step - loss: 0.1265 - accuracy: 0.9623 - val_loss: 0.1683 - val_accuracy: 0.9555
Epoch 6/20
24/24 [==============================] - 1s 54ms/step - loss: 0.1047 - accuracy: 0.9669 - val_loss: 0.1501 - val_accuracy: 0.9613
Epoch 7/20
24/24 [==============================] - 1s 54ms/step - loss: 0.0930 - accuracy: 0.9708 - val_loss: 0.1319 - val_accuracy: 0.9658
Epoch 8/20
24

In [14]:
model = Sequential([
    
     # reshape 28 row * 28 column data to 28*28 rows
    Flatten(input_shape=(28, 28)),
    
      # dense layer 1
    Dense(512, activation='relu'),
    BatchNormalization(),  
    Dropout(0.2),
    # dense layer 2
    Dense(128, activation='relu'), 
    BatchNormalization(),  
    Dropout(0.2),
      # output layer
    Dense(10, activation='softmax'),
])

optimizer = GLFGD(learning_rate = 0.08, momentum = 0.99, nesterov = True)
model.compile(optimizer=optimizer,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

history2 = model.fit(x_train, y_train, epochs=20, 
          batch_size=2000, 
          validation_split=0.2)

Epoch 1/20
24/24 [==============================] - 2s 60ms/step - loss: 0.5624 - accuracy: 0.8350 - val_loss: 0.5171 - val_accuracy: 0.8393
Epoch 2/20
24/24 [==============================] - 1s 51ms/step - loss: 0.2303 - accuracy: 0.9297 - val_loss: 0.3882 - val_accuracy: 0.8921
Epoch 3/20
24/24 [==============================] - 1s 54ms/step - loss: 0.1688 - accuracy: 0.9486 - val_loss: 0.3508 - val_accuracy: 0.9189
Epoch 4/20
24/24 [==============================] - 1s 55ms/step - loss: 0.1399 - accuracy: 0.9581 - val_loss: 0.2548 - val_accuracy: 0.9468
Epoch 5/20
24/24 [==============================] - 1s 58ms/step - loss: 0.1169 - accuracy: 0.9634 - val_loss: 0.2089 - val_accuracy: 0.9548
Epoch 6/20
24/24 [==============================] - 1s 53ms/step - loss: 0.0997 - accuracy: 0.9695 - val_loss: 0.1604 - val_accuracy: 0.9632
Epoch 7/20
24/24 [==============================] - 1s 54ms/step - loss: 0.0872 - accuracy: 0.9728 - val_loss: 0.1421 - val_accuracy: 0.9665
Epoch 8/20
24

In [15]:
model = Sequential([
    
     # reshape 28 row * 28 column data to 28*28 rows
    Flatten(input_shape=(28, 28)),
    
      # dense layer 1
    Dense(512, activation='relu'),
    BatchNormalization(),  
    Dropout(0.2),
    # dense layer 2
    Dense(128, activation='relu'), 
    BatchNormalization(),  
    Dropout(0.2),
      # output layer
    Dense(10, activation='softmax'),  
])

optimizer = Adam(learning_rate = 0.001)
model.compile(optimizer=optimizer,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

history3 = model.fit(x_train, y_train, epochs=20, 
          batch_size=2000, 
          validation_split=0.2)

Epoch 1/20
24/24 [==============================] - 2s 57ms/step - loss: 0.6066 - accuracy: 0.8148 - val_loss: 0.9841 - val_accuracy: 0.7702
Epoch 2/20
24/24 [==============================] - 2s 73ms/step - loss: 0.2167 - accuracy: 0.9371 - val_loss: 0.8443 - val_accuracy: 0.9028
Epoch 3/20
24/24 [==============================] - 2s 63ms/step - loss: 0.1567 - accuracy: 0.9546 - val_loss: 0.7383 - val_accuracy: 0.9310
Epoch 4/20
24/24 [==============================] - 2s 62ms/step - loss: 0.1247 - accuracy: 0.9636 - val_loss: 0.6361 - val_accuracy: 0.9398
Epoch 5/20
24/24 [==============================] - 1s 58ms/step - loss: 0.1002 - accuracy: 0.9708 - val_loss: 0.5199 - val_accuracy: 0.9496
Epoch 6/20
24/24 [==============================] - 1s 60ms/step - loss: 0.0822 - accuracy: 0.9766 - val_loss: 0.4282 - val_accuracy: 0.9578
Epoch 7/20
24/24 [==============================] - 2s 63ms/step - loss: 0.0690 - accuracy: 0.9801 - val_loss: 0.3262 - val_accuracy: 0.9578
Epoch 8/20
24

In [16]:
model = Sequential([
    
     # reshape 28 row * 28 column data to 28*28 rows
    Flatten(input_shape=(28, 28)),
    
      # dense layer 1
    Dense(512, activation='relu'),
    BatchNormalization(),  
    Dropout(0.2),
    # dense layer 2
    Dense(128, activation='relu'), 
    BatchNormalization(),  
    Dropout(0.2),
      # output layer
    Dense(10, activation='softmax'),  
])

optimizer = SGD(learning_rate = 0.01, momentum = 0.99, nesterov = True)
model.compile(optimizer=optimizer,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

history4 = model.fit(x_train, y_train, epochs=20, 
          batch_size=2000, 
          validation_split=0.2)

Epoch 1/20
24/24 [==============================] - 2s 58ms/step - loss: 0.9733 - accuracy: 0.7030 - val_loss: 0.9364 - val_accuracy: 0.7168
Epoch 2/20
24/24 [==============================] - 1s 54ms/step - loss: 0.3672 - accuracy: 0.8903 - val_loss: 0.6808 - val_accuracy: 0.7855
Epoch 3/20
24/24 [==============================] - 1s 57ms/step - loss: 0.2895 - accuracy: 0.9124 - val_loss: 0.3675 - val_accuracy: 0.8932
Epoch 4/20
24/24 [==============================] - 1s 52ms/step - loss: 0.2419 - accuracy: 0.9267 - val_loss: 0.2580 - val_accuracy: 0.9223
Epoch 5/20
24/24 [==============================] - 1s 53ms/step - loss: 0.2084 - accuracy: 0.9369 - val_loss: 0.2063 - val_accuracy: 0.9381
Epoch 6/20
24/24 [==============================] - 1s 57ms/step - loss: 0.1795 - accuracy: 0.9453 - val_loss: 0.1801 - val_accuracy: 0.9441
Epoch 7/20
24/24 [==============================] - 1s 53ms/step - loss: 0.1595 - accuracy: 0.9510 - val_loss: 0.1600 - val_accuracy: 0.9517
Epoch 8/20
24